In [36]:
#!pip install azure-cognitiveservices-vision-customvision
#!pip install --upgrade pandas

  Found existing installation: pandas 0.23.0
    Uninstalling pandas-0.23.0:


fastparquet 0.1.6 requires pytest-runner, which is not installed.
Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\anaconda\\lib\\site-packages\\pandas-0.23.0.dist-info\\installer'
Consider using the `--user` option or check the permissions.

You are using pip version 18.0, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [37]:
print(pd.__version__)

0.23.0


In [5]:
import cv2, json, os
import matplotlib.pyplot as plt
import numpy as np
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry, Region
from PIL import Image,ImageDraw
import pandas as pd
from matplotlib import image as mpimg

In [49]:
%run common.ipynb

In [17]:
images = 'E:/PawseyData/archive/FDFML/frames/'

labels = '.\\labelmetadata'

In [18]:
frame_labels = pd.read_csv(labels+'\\frame_metadata.csv')

In [19]:
frame_labels.head(5)

,uid,file_name,x0,y0,x1,y1,family,genus,species
0,1,A000001_L.avi.5107.png,806,371,922,448,Scaridae,Chlorurus,capistratoides
1,2,A000001_R.avi.4902.png,1388,355,1508,435,Scaridae,Chlorurus,capistratoides
2,3,A000001_L.avi.5174.png,912,280,994,335,Scaridae,Chlorurus,capistratoides
3,4,A000001_R.avi.4969.png,1600,255,1682,310,Scaridae,Chlorurus,capistratoides
4,5,A000001_L.avi.5194.png,811,472,979,584,Acanthuridae,Acanthurus,leucocheilus


In [20]:
import os
frames_list = []
frame_names = list(frame_labels['file_name'])
for filename in os.listdir(images):
    root, ext = os.path.splitext(filename)
    frames_list.append(root+ext)
    
    if root+'.png' in frame_names:
        frame_labels.loc[frame_labels['file_name'] == root+'.png', ['file_name']]= root+ext

In [22]:
test_labels = frame_labels[:100]

In [23]:
def normalizeregions(cords, filename):
    l,t,r,b = cords
    #normalize values
    img = mpimg.imread(os.path.join(images, filename))
    img_h, img_w, img_ch = img.shape
    l = l / img_w
    r = r / img_w
    t = t/img_h
    b = b / img_h

    w = r-l
    h = b-t
    return [l, t, w, h]

In [53]:
normalized_regions = {}


for index, row in frame_labels[:1000].iterrows():
    frames = {}
    filename = row['file_name']
    label = row['species']
    #image = Image.open(os.path.join(images, filename))
    cordinates = normalizeregions(np.array(row[['x0','y0','x1','y1']]).tolist(), filename)
    
    if label not in normalized_regions:
        frames[filename] = [cordinates]
        normalized_regions[label] = frames
        
    elif filename not in normalized_regions[label]:
        normalized_regions[label][filename] = [cordinates]
    else:
        normalized_regions[label][filename].append(cordinates)
    
    
    
#normalized_regions        

In [54]:
len(normalized_regions)

78

In [50]:
project_name = 'KakaduFishAIDev'
trainer = CustomVisionTrainingClient(custom_vision_training_key, endpoint=custom_vision_endpoint)

In [51]:
projects = trainer.get_projects()
project_names = [project.name for project in projects]

project = None

if not project_name in project_names:
    print('Project {0} not found. Creating...'.format(project_name))
    object_detection_domain_id = next(domain.id for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")
    project = trainer.create_project(name=project_name, domain_id=object_detection_domain_id)
    print('Created project {}.'.format(project_name))
else:
    project = [project for project in projects if project.name == project_name][0]

In [55]:
project.name

'KakaduFishAIDev'

In [56]:
project_tags = trainer.get_tags(project.id)
project_tag_names = [project_tag.name for project_tag in project_tags]

tags = {}
    
for fish_tag in normalized_regions.keys():
    if not fish_tag in project_tag_names:
        print('Creating tag {0}...'.format(fish_tag))
        tags[fish_tag] = trainer.create_tag(project.id, fish_tag)
    else:
        print('Fetching tag {0}...'.format(option))
        project_tag = [project_tag for project_tag in project_tags if project_tag.name == fish_tag][0]
        tags[fish_tag] = project_tag

Creating tag capistratoides...
Creating tag leucocheilus...
Creating tag reticulatus...
Creating tag fasciatus...
Creating tag pyroferus...
Creating tag scopas...
Creating tag digrammus...
Creating tag coelestis...
Creating tag weberi...
Creating tag multifasciatus...
Creating tag striatus...
Creating tag undulatus...
Creating tag chrysopterum...
Creating tag trimaculatus...
Creating tag viridescens...
Creating tag vlamingii...
Creating tag melampygus...
Creating tag cinerascens...
Creating tag albimarginata...
Creating tag hexacanthus...
Creating tag macularis...
Creating tag rogaa...
Creating tag teres...
Creating tag bohar...
Creating tag dimidiatus...
Creating tag gibbus...
Creating tag vari...
Creating tag microrhinos...
Creating tag diacanthus...
Creating tag imperator...
Creating tag flavissimus...
Creating tag louti...
Creating tag doliatus...
Creating tag lunare...
Creating tag urodeta...
Creating tag atkinsoni...
Creating tag amblycephalum...
Creating tag temminckii...
Creati

In [57]:
for fish_tag in normalized_regions.keys():
    for file_name in normalized_regions[fish_tag].keys():
        tagged_images_with_regions = []
        print('Uploading {0}...'.format(file_name))
        regions = []
        for normalized_region in normalized_regions[fish_tag][file_name]:
            x, y, w, h = normalized_region
            tag_id = tags[fish_tag].id
            regions.append(Region(tag_id=tag_id, left=x, top=y, width=w, height=h) )

        with open(os.path.join(images, file_name), mode="rb") as image_contents:
            tagged_images_with_regions.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), regions=regions))
    
        upload_result = trainer.create_images_from_files(project.id, images=tagged_images_with_regions)
        
        if not upload_result.is_batch_successful:
            print("Image batch upload failed.")
            for image in upload_result.images:
                print("Image status: ", image.status)

Uploading A000001_L.avi.5107.png-17006...
Uploading A000001_R.avi.4902.png-17005...
Uploading A000001_L.avi.5174.png-17004...
Uploading A000001_R.avi.4969.png-17003...
Uploading A000002_L.avi.19113.png-16729...
Uploading A000002_R.avi.19433.png-16727...
Uploading A000002_L.avi.19127.png-16726...
Uploading A000002_R.avi.19447.png-16724...
Uploading A000003_L.avi.22693.png-16549...
Uploading A000003_R.avi.22465.png-16546...
Uploading A000003_L.avi.22717.png-16547...
Uploading A000003_R.avi.22489.png-16544...
Uploading A000004_L.avi.14309.png-16435...
Uploading A000004_R.avi.13956.png-16432...
Uploading A000005_L.avi.5101.png-16284...
Uploading A000005_R.avi.4879.png-16283...
Uploading A000005_L.avi.5136.png-16130...
Uploading A000005_R.avi.4914.png-16129...
Uploading A000001_L.avi.5194.png-17002...
Uploading A000001_R.avi.4989.png-17001...
Uploading A000001_L.avi.5401.png-17000...
Uploading A000001_R.avi.5196.png-16998...
Uploading A000001_L.avi.5422.png-16999...
Uploading A000001_R.avi.

Uploading A000001_R.avi.38778.png-16892...
Uploading A000001_L.avi.38990.png-16893...
Uploading A000001_R.avi.38785.png-16890...
Uploading A000001_L.avi.39011.png-16891...
Uploading A000001_R.avi.38806.png-16888...
Uploading A000001_L.avi.39053.png-16889...
Uploading A000001_R.avi.38848.png-16886...
Uploading A000001_L.avi.39064.png-16887...
Uploading A000001_R.avi.38859.png-16885...
Uploading A000001_L.avi.39069.png-16884...
Uploading A000001_R.avi.38864.png-16883...
Uploading A000001_L.avi.39079.png-16882...
Uploading A000001_R.avi.38874.png-16881...
Uploading A000002_L.avi.45824.png-16693...
Uploading A000002_R.avi.46144.png-16691...
Uploading A000003_L.avi.82025.png-16462...
Uploading A000003_R.avi.81797.png-16461...
Uploading A000004_L.avi.67896.png-16359...
Uploading A000004_R.avi.67543.png-16357...
Uploading A000004_L.avi.67921.png-16356...
Uploading A000004_R.avi.67568.png-16355...
Uploading A000004_L.avi.67947.png-16354...
Uploading A000004_R.avi.67594.png-16353...
Uploading A

Uploading A000003_L.avi.73555.png-16504...
Uploading A000003_R.avi.73327.png-16503...
Uploading A000003_L.avi.73676.png-16502...
Uploading A000003_R.avi.73448.png-16500...
Uploading A000003_L.avi.73219.png-16477...
Uploading A000003_R.avi.72991.png-16474...
Uploading A000003_L.avi.73332.png-16475...
Uploading A000003_R.avi.73104.png-16472...
Uploading A000003_L.avi.73407.png-16473...
Uploading A000003_R.avi.73179.png-16470...
Uploading A000003_L.avi.73415.png-16471...
Uploading A000003_R.avi.73187.png-16469...
Uploading A000003_L.avi.73432.png-16468...
Uploading A000003_R.avi.73204.png-16467...
Uploading A000003_L.avi.73553.png-16466...
Uploading A000003_R.avi.73325.png-16465...
Uploading A000003_L.avi.73608.png-16464...
Uploading A000003_R.avi.73380.png-16463...
Uploading A000003_L.avi.73459.png-16449...
Uploading A000003_R.avi.73231.png-16447...
Uploading A000004_L.avi.14863.png-16433...
Uploading A000004_R.avi.14510.png-16430...
Uploading A000004_L.avi.14966.png-16431...
Uploading A

Uploading A000001_R.avi.77954.png-16760...
Uploading A000001_L.avi.79099.png-16761...
Uploading A000001_R.avi.78894.png-16758...
Uploading A000002_L.avi.40177.png-16700...
Uploading A000002_R.avi.40497.png-16699...
Uploading A000002_L.avi.39984.png-16637...
Uploading A000002_R.avi.40304.png-16634...
Uploading A000002_L.avi.40194.png-16635...
Uploading A000002_R.avi.40514.png-16632...
Uploading A000005_L.avi.5206.png-16282...
Uploading A000005_R.avi.4984.png-16281...
Uploading A000001_L.avi.79789.png-16759...
Uploading A000001_R.avi.79584.png-16756...
Uploading A000002_L.avi.17054.png-16731...
Uploading A000002_R.avi.17374.png-16728...
Uploading A000002_L.avi.17200.png-16651...
Uploading A000002_R.avi.17520.png-16649...
Uploading A000003_L.avi.61343.png-16536...
Uploading A000003_R.avi.61115.png-16534...
Uploading A000003_L.avi.61166.png-16479...
Uploading A000003_R.avi.60938.png-16476...
Uploading A000003_L.avi.62046.png-16453...
Uploading A000003_R.avi.61818.png-16450...
Uploading A00

Uploading A000004_R.avi.88698.png-16335...
Uploading A000004_L.avi.29412.png-16300...
Uploading A000004_R.avi.29059.png-16299...
Uploading A000005_L.avi.20184.png-16246...
Uploading A000005_R.avi.19962.png-16245...
Uploading A000005_L.avi.16456.png-16254...
Uploading A000005_R.avi.16234.png-16253...
Uploading A000005_L.avi.22373.png-16242...
Uploading A000005_R.avi.22151.png-16241...
